In [ ]:
from PIL import Image
from torch.utils.data import Dataset
import os
from glob import glob
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchsummary import summary


# Dataset Class for Setting up the data loading process
# Stuff to fill in this script: _init_transform()
class inaturalist(Dataset):
    def __init__(self, root_dir, mode='train', transform=True):
        self.data_dir = root_dir
        self.mode = mode
        self.transforms = transform
        self._init_dataset()
        if transform:
            self._init_transform()

    def _init_dataset(self):
        self.files = []
        self.labels = []
        dirs = sorted(os.listdir(os.path.join(self.data_dir, 'train')))
        if self.mode == 'train':
            for dir in range(len(dirs)):
                files = sorted(glob(os.path.join(self.data_dir, 'train', dirs[dir], '*.jpg')))
                self.labels += [dir] * len(files)
                self.files += files
        elif self.mode == 'val':
            for dir in range(len(dirs)):
                files = sorted(glob(os.path.join(self.data_dir, 'val', dirs[dir], '*.jpg')))
                self.labels += [dir] * len(files)
                self.files += files
        else:
            print("No Such Dataset Mode")
            return None

    def _init_transform(self):
        self.transform = transforms.Compose([
            transforms.CenterCrop(32),
            transforms.ToTensor(),
        ])

    def __getitem__(self, index):
        img = Image.open(self.files[index]).convert('RGB')
        label = self.labels[index]

        if self.transforms:
            img = self.transform(img)

        label = torch.tensor(label, dtype=torch.long)

        return img, label

    def __len__(self):
        return len(self.files)
    

import torch


#Class to define the model which we will use for training
#Stuff to fill in: The Architecture of your model, the forward function to define the forward pass
# NOTE!: You are NOT allowed to use pretrained models for this task

class Classifier(nn.Module):
    def __init__(self, n_classes):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # output: 64 x 16 x 16

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # output: 128 x 8 x 8

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # output: 256 x 4 x 4

            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, n_classes))

    def forward(self, x):
        
        return self.network(x)

   




# Sections to Fill: Define Loss function, optimizer and model, Train and Eval functions and the training loop

############################################# DEFINE HYPERPARAMS #####################################################
# Feel free to change these hyperparams based on your machine's capactiy
batch_size = 5
epochs = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

############################################# DEFINE DATALOADER #####################################################
trainset = inaturalist(root_dir='../input/nature-12k/inaturalist_12K', mode='train')
valset = inaturalist(root_dir='../input/nature-12k/inaturalist_12K', mode='val')
train_set=[]

for i in range(10):
    for j in range(100):
        k=(1000*i)+j
        train_set.append(trainset[k])


trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
valloader = DataLoader(valset, batch_size=1, shuffle=False, num_workers=4)

################################### DEFINE LOSS FUNCTION, MODEL AND OPTIMIZER ######################################
criterion = nn.CrossEntropyLoss()

model = Classifier(n_classes=10)

optimize = optim.Adam


################################### CREATE CHECKPOINT DIRECTORY ####################################################

# NOTE: If you are using Kaggle to train this, remove this section. Kaggle doesn't allow creating new directories.
#checkpoint_dir = 'checkpoints'
#if not os.path.isdir(checkpoint_dir):
 #   os.makedirs(checkpoint_dir)


#################################### HELPER FUNCTIONS ##############################################################

def get_model_summary(model, input_tensor_shape):
    summary(model, input_tensor_shape)
    

def accuracy(y_pred, y):
    _, predicted = torch.max(y_pred.data, 1)
    total = y.size(0)
    correct = (predicted == y).sum().item()
    return correct / total


def train(model, dataset, optimizer, criterion, device):
    '''
    Write the function to train the model for one epoch
    Feel free to use the accuracy function defined above as an extra metric to track
    '''
    s=0
    tot_loss=0
    tot_metric=0
    for xb,yb in dataset:
        xb=xb.to(device)
        yb=yb.to(device)
        pred=model(xb)
     
        loss=F.cross_entropy(pred, yb)
        s+=len(xb)
        tot_loss+=(loss*len(xb))
        acc=accuracy(pred, yb)
        tot_metric+=(acc*len(xb))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        
    return tot_loss,tot_metric,s
   


def evalu(model, dataset, criterion, device):
    '''
    Write the function to validate the model after each epoch
    Feel free to use the accuracy function defined above as an extra metric to track
    '''
    with torch.no_grad():
        s=[]
        tot_loss=[]
        tot_metric=[]
        for xb,yb in dataset:
                xb=xb.to(device)
                yb=yb.to(device)
                pred=model(xb)
                loss=criterion(pred, yb)
                s+=len(xb)
                tot_loss+=(loss*len(xb))
                acc=accuracy(pred, yb)
                tot_metric+=(acc*len(xb))
                torch.cuda.empty_cache()
        return tot_loss,tot_metric,s






def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


################################################### TRAINING #######################################################
# Get model Summary
model=model.to(device, non_blocking=True);
#get_model_summary(model, (3, 256, 256))

# Training and Validation
best_valid_loss = float('inf')
def fit(learning_rate):
  
   
    op=optimize(model.parameters(),learning_rate)
    for epoch in range(epochs):
        start_time = time.monotonic()
        
        
        '''
        Insert code to train and evaluate the model (Hint: use the functions you previously made :P)
        Also save the weights of the model in the checkpoint directory
        '''
        tloss,tacc,ts=train(model, trainloader,op,criterion, device )
        print("loss:",(tloss/ts),"Accuracy:",(tacc/ts))
        
        #t1loss,t1acc,t1s=evalu(model,valloader,loss_func,device)
        #print("loss:",(t1loss/t1s),"accuracy:",(t1acc/t1s))


        
print("OVERALL TRAINING COMPLETE")
   


In [ ]:
!pip3 install torchsummary

In [ ]:
learning_rate = 0.0001
for i in range(10):
    print(i)
    fit(learning_rate)
print("comp")